# Five-Room Dungeon Generator

The Five-Room Dungeon Generator provides GMs with a way to quickly and easily generate a custom five-room dungeon that can be incorporated into their campaign.  Simply provide the notebook with a concept and the notebook will do the rest.

> [!IMPORTANT]
> You will need an [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .NET Interactive.

## Step 1 - Define Your Concept

Define your five-room dungeon concept.

In [83]:
var concept = @"
You are a game master for a Dungeons and Dragons campaign.
Please create a five-room dungeon for a party of 5 level 3 characters.
The five-room dungeon should be of medium difficulty and take place in the town of Hammersend.
The party must deal with a powerful cult that has taken control of the town of Hammersend.
The cult is comprised of acolytes and clerics who worship ancient sea witches.
The cult has formed a partnership with one of the player's brothers, who has been corrupted by the cult's influence.
The party must infiltrate the cult's evil lair, an abandoned lighthouse.
The cult will summon a tentacle monster to destroy the party and the town if they are not stopped.

Ensure that the output includes:
* the five-room dungeon outline, 
* descriptive text for each room in the outline, 
* a list of NPCs and monsters that the party will encounter, 
* a rolltable of possible loot that the party can find.

Output options:
* Please output the generated information into a single consolidated markdown document.
* please do not format results into json, csv, or other structured data formats
";

## Step 2 - Initial Configuration


Choose whether you wish to use the OpenAI or Azure OpenAI service.

In [98]:
bool useAzureOpenAI = false;

Execute the following code which will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

In [99]:
#!import config/Settings.cs

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you're using OpenAI and need to set the Org Id
await Settings.AskOrg(useAzureOpenAI);

Settings: OK: AI model configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


If you want to reset the configuration and start again, please uncomment and run the code below.
You can also edit the [config/settings.json](config/settings.json) manually if you prefer.

In [100]:
#!import config/Settings.cs

// Uncomment this line to reset your settings and delete the file from disk.
// Settings.Reset();

## Step 3 - Instantiate Services

Set up the Microsoft Semantic AI Kernel that will use for generative operations.

In [101]:

#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview"
#r "nuget: System.Numerics.Tensors, 8.0.0"
#r "nuget: SkiaSharp, 2.88.3"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.Numerics.Tensors;

#pragma warning disable SKEXP0001, SKEXP0010

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
{
    builder.AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
    
else
{
    builder.AddOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId);
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

Installed Packages Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview SkiaSharp, 2.88.3 System.Numerics.Tensors, 8.0.0

Import custom plugins from plugin directory.

In [102]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");

var kernel = builder.Build();
var pluginFunctions = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "FiveRoomDungeonPlugin"));

Set up the Handlebars Planner that will be used for orchestrating generative operations.

In [103]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

## Step 4 - Develop an Execution Plan

Have the planner generate an executiong plan based on the provided concept and kernel plugins.

In [93]:
#pragma warning disable SKEXP0060

// Generate a plan for the concept
var executionPlan  = await planner.CreatePlanAsync(kernel, concept);
Console.WriteLine(executionPlan);

{{!-- Step 0: Extract key values --}}
{{set "gameSystem" "Dungeons and Dragons"}}
{{set "inputContext" "The town of Hammersend has been taken over by a cult that worships ancient sea witches. This cult consists of acolytes and clerics and has influenced one of the player's brothers. The lair is an abandoned lighthouse where they plan to summon a tentacle monster to destroy the town if not stopped."}}

{{!-- Step 1: Generate a five-room dungeon outline --}}
{{set "dungeonOutline" (FiveRoomDungeonPlugin-FiveRoomDungeon game_system=gameSystem input_context=inputContext)}}

{{!-- Step 2: Generate descriptive text for each room --}}
{{set "descriptionRoom1" (FiveRoomDungeonPlugin-RoomDescription room_number="1" input_context=dungeonOutline)}}
{{set "descriptionRoom2" (FiveRoomDungeonPlugin-RoomDescription room_number="2" input_context=dungeonOutline)}}
{{set "descriptionRoom3" (FiveRoomDungeonPlugin-RoomDescription room_number="3" input_context=dungeonOutline)}}
{{set "descriptionRoom4" (Fi

## Step 5 - Execute the Plan
Execute the plan and generate the five-room dungeon outline and room descriptions.

In [94]:
#pragma warning disable SKEXP0060

var executionPlanResult = await executionPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine(executionPlanResult.ToString());

# Five-Room Dungeon: Infiltrating the Cult of Ancient Sea Witches

## Dungeon Outline
## Room 1: Entrance and Guardian
* **Objective:** Create an initial challenge.
* **Design:** The entrance to the abandoned lighthouse is guarded by two cultist acolytes and a magically sealed door. The acolytes are chanting an incantation that maintains the seal. Players must either defeat the acolytes or find a way to disrupt their chant to break the seal and gain entry.

## Room 2: Puzzle or Roleplaying Challenge
* **Objective:** Engage players with non-combat interaction.
* **Design:** Inside the lighthouse, the players find a room filled with ancient nautical charts and a large, intricate model of the sea. To proceed, they must solve a puzzle involving the placement of miniature ships on the model to match a specific constellation pattern. Alternatively, they could find a hidden journal that provides clues to the correct arrangement.

## Room 3: Trick or Setback
* **Objective:** Introduce a compli

## Step 6 - Structure the Data

The generated content will be more useful to us if we can structure it into a data format that can be consumed by other systems.

In [114]:
// Construct arguments
var arguments = new KernelArguments() { ["input_context"] = markdownContent };

// Define the POCO classes
public class NPC
{
    public string name { get; set; }
    public string appearance { get; set; }
    public string personality { get; set; }
    public string role { get; set; }
}

public class Entry
{
    public int roll { get; set; }
    public string item { get; set; }
    public string description_basic { get; set; }
    public string description_intermediate { get; set; }
    public string description_advanced { get; set; }
}

public class RollTable
{
    public string description { get; set; }
    public List<Entry> entries { get; set; }
}

public class LootTable
{
    public RollTable rolltable { get; set; }
}

public class FiveRoomDungeon
{
    public string title { get; set; }
    public string introduction { get; set; }
    public string plot_twist { get; set; }
    public List<Room> rooms { get; set; }
}

public class Room
{
    public int room_number { get; set; }
    public string title { get; set; }
    public string objective { get; set; }
    public string design { get; set; }
    public string description { get; set; }
}

// Create the output directory
var outputDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "output");
if (!Directory.Exists(outputDirectory))
{
    Directory.CreateDirectory(outputDirectory);
}

// Parse the Five-Room Dungeon markdown content into JSON
var jsonContent5RD = await kernel.InvokeAsync(pluginFunctions["JSONFormatFiveRoomDungeon"], arguments);
var jsonContent5RDString = jsonContent5RD.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.json"), jsonContent5RDString);
FiveRoomDungeon fiveRoomDungeon = JsonSerializer.Deserialize<FiveRoomDungeon>(jsonContent5RDString);

// Parse the Loot markdown content into JSON
var jsonContentLoot = await kernel.InvokeAsync(pluginFunctions["JSONFormatLootTable"], arguments);
var jsonContentLootString = jsonContentLoot.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "loot-table.json"), jsonContentLootString);
LootTable lootTable = JsonSerializer.Deserialize<LootTable>(jsonContentLootString);

// Parse the NPC markdown content into JSON
var jsonContentNPCs = await kernel.InvokeAsync(pluginFunctions["JSONFormatNPCs"], arguments);
var jsonContentNPCsString = jsonContentNPCs.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "npcs.json"), jsonContentNPCsString);
List<NPC> npcs = JsonSerializer.Deserialize<List<NPC>>(jsonContentNPCsString);

## Step 7 - Use the Structured Data

Now that we have structured data, we can programmatically iterate through objects of interest and perform further action.  In this case, we'll be generating token images for each NPC.

In [116]:
#pragma warning disable SKEXP0001

var prompt = "A photo-realistic portrait of an NPC with the following description: ";

// Create an httpClient
var httpClient = new HttpClient();

// // Output the deserialized objects
// foreach (var npc in npcs)
// {
//     // Use DALL-E 3 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
//     var imageUrl = await dallE.GenerateImageAsync(prompt + npc.appearance, 1024, 1024);

//     // Download the image to a local file
//     using (Stream stream = await httpClient.GetStreamAsync(imageUrl))
//     using (MemoryStream memStream = new MemoryStream())
//     {
//         await stream.CopyToAsync(memStream);
//         File.WriteAllBytes(Path.Combine(outputDirectory, $"{npc.name}.png"), memStream.ToArray());
//     };
// }

// Construct the markdown content
var markdownContent = new StringBuilder();
markdownContent.AppendLine($"# {fiveRoomDungeon.title}");
markdownContent.AppendLine();
markdownContent.AppendLine(fiveRoomDungeon.introduction);
markdownContent.AppendLine();
markdownContent.AppendLine("## Plot Twist");
markdownContent.AppendLine(fiveRoomDungeon.plot_twist);
markdownContent.AppendLine();
markdownContent.AppendLine("## Rooms");
foreach (var room in fiveRoomDungeon.rooms)
{
    markdownContent.AppendLine($"### Room {room.room_number}: {room.title}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Objective**: {room.objective}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Design**: {room.design}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Description**: {room.description}");
    markdownContent.AppendLine();
}
markdownContent.AppendLine("## NPCs");
foreach (var npc in npcs)
{
    markdownContent.AppendLine($"### {npc.name}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Appearance**: {npc.appearance}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Personality**: {npc.personality}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Role**: {npc.role}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"![{npc.name}](./{npc.name}.png)");
    markdownContent.AppendLine();
}
markdownContent.AppendLine("## Loot Table");
markdownContent.AppendLine(lootTable.rolltable.description);
markdownContent.AppendLine();
markdownContent.AppendLine("| Roll | Item | Basic Description | Intermediate Description | Advanced Description |");
markdownContent.AppendLine("|------|------|-------------------|-------------------------|----------------------|");
if (lootTable.rolltable.entries != null)
{
    foreach (var entry in lootTable.rolltable.entries)
    {
        markdownContent.AppendLine($"| {entry.roll} | {entry.item} | {entry.description_basic} | {entry.description_intermediate} | {entry.description_advanced} |");
    }
}

// Output the markdown content to a file
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.md"), markdownContent.ToString());
